In [93]:
import collections
import random
import time
import pandas as pd
import sys
# Alphabet Characters
Alphabet_Characters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S'
            ,'T','U','V','W','X','Y','Z'];
# Character Positions
Positions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25];

# Create a mapping of characters to position
def Alphabet_Position_Mapping():
    Alphabet = collections.defaultdict(list)
    for i in range(0,len(Alphabet_Characters)):
        Alphabet[i].append(Alphabet_Characters[i])
        Alphabet[i].append(Positions[i])
    return Alphabet   

Alphabet = Alphabet_Position_Mapping();

In [94]:
Plaintext_Alphabet = collections.defaultdict(list)
for n in range (26):
    Plaintext_Alphabet[Alphabet[n][0]] = Alphabet[n][1]

# Overview


The cipher uses three private keys. Key1 is a the result of a hash function of the input message and addition of the hash of the current time in UNIX time, this was done as only using the hash of the message would allow for the same private key to be created multiple times. With the addition of the hash function of the current unix time, which changes every millisecond, every key created by the cipher will be unique, except for cases in which the cipher is used at exactly the same time to encrypt the same message, which is practically impossible.

Key2 is used to record the index in the message in which a white-space occurs. The white-space is then replaced with a random number of randomly selected characters. The number of characters that replace each space is stored in Key3.

## Creating The Key

Input:  Plaintext message

Output:  Key

•Assign Key = Hash(Current UNIX Time)

•Convert Key (Currently a string) To An Integer

•Assign Key as:  Key + Hash(Plaintext Message)

•Convert the key back to a string

•Return Key

## Encryption

Input:  Plaintext message

Return:  Key1,Key2,Key3,Encrypted Message

•Key1 = CreateKey(Plaintext Message)

•Count = 0 ; used as index to plaintext message

•While End of The Plaintext Message is Not Reached DO:

    If PlainText Message[Count] IS NOT Whitespace, DO:

    For Each Element In The Key DO:
        New Position = Current Characters Position In Alphabet + Current Element In The Key
            If New Position GREATER THAN Length Of The Alphabet, DO:

                Append Count To Key2
                New Position = Negative(Length Of The Alphabet) + (Current CharactersPosition In Alphabet + Current Element In The Key)
                Encypted Message[Count] = Plaintext Alphabet[New Position]

    Else New Position[Count] IS Whitespace, DO:

        Generate Random Interger, Num Randoms
        Append Num Randoms To Key3
        X = 1 :  Num Randoms, DO: 
            Random Number between 0 And Length Of The Alphabet:  New
            Position Alphabet Character[New Position] To the Encrypted Message
            
    If Count = Length Of Plaintext Message, DO: Return Key1,Key2,Key3,Encrypted Message

## Decryption

Input:  Key1,Key2,Key3,Encrypted Message

Return:  Decrypted Message

•Count = 0 ; used as index to encrypted message

•Key Count = 0 ; used as index to Keys 2 and 3

•While Message Not Decrypted:

•For Each Element In The Key DO:

    If the Key Counter is In Key2 DO:
    
        Plaintext Index = length Of Alphabet + CipherText characters Index In Alphabet
        
    If the Key Counter is In Key3 DO:
        
        Add Single Whitespace to Decrypted Message
        Increment Count By Value In Key3[Key Counter]
        Increment Key Counter By 1
            
    If Key Counter Is Not In Key2 Or Key3 DO:
        
        PlainText Index = CipherText Character Position In Alphabet - Current Element of Key
        Add Plaintext Character from The Alphabet at Position PlainText Index•Increment Count And Key Count By 1

## Result Of Encrypting The Same Message Twice

In [95]:

Input = ['Sussex','Sussex']
Key1 = [3965300635186948835,4894608754009758453]
Encrypted_Message = ['VEYXHX','WDCWKX']
Decrypted_Message = ['sussex','sussex']
# Create dataframe
df = pd.DataFrame(Input)
# Lets change the name of the column
df.columns = ['Input']
df['Key1'] = Key1
df['Encypted Message'] = Encrypted_Message
df['Decypted Message'] = Decrypted_Message

df

,Input,Key1,Encypted Message,Decypted Message
0,Sussex,3965300635186948835,VEYXHX,sussex
1,Sussex,4894608754009758453,WDCWKX,sussex


### Result Of Encrypting The Same Message Twice At The Same Time (As close as possible at leased)

the best i could manage to get was 0.00016689300537109375 Epochs difference, about a thousandth of a second,the cipher still produces vastly different Key1 values and the encrypted messages bare no resemblance

In [96]:
# Our small data set
Input = ['Sussex not susex','Sussex not susex']
Timestamp = [1544607651.106532,1544608433.688006]
Key1 = [5489625615527348109,1941271845566194205]
Encrypted_Message = ['XYBCKZJLPSSCHAXYBCKZ','TEWTGFMOXXJTEWTGF']
# Create dataframe
df = pd.DataFrame(Input)
# Lets change the name of the column
df.columns = ['Input']
df['Timestamp'] = Timestamp
df['Key1'] = Key1
df['Encypted Message'] = Encrypted_Message


df

,Input,Timestamp,Key1,Encypted Message
0,Sussex not susex,1.544608e+09,5489625615527348109,XYBCKZJLPSSCHAXYBCKZ
1,Sussex not susex,1.544608e+09,1941271845566194205,TEWTGFMOXXJTEWTGF


### Role Of Key2

Key2  Records  the  index  at  which  a  wraparound  occurred;  it  allows  the  decryption  function  toobtain the correct alphabet index of the plain text.  Below demonstrates what would happen shouldan attacker have access to the encrypted message, Keys 1 and 3, but not Key2.

Notice  that  Key2  in  [0,5]  and  the  missing  values  of  the  decrypted  message,the decryption function is unable to obtain an index for characters at indices 0 and 5 as they are where a wrap-around of the alphabet occurred.  This could be seen as aweakness in the cipher as an attacker would be able to decrpyt parts of the message without the fullkey.

Rows  3  and  4  show  the  result  of  trying  to  decrypt  a  longer  word, ’Internationalization’,  with  and without access to  key2.  Row 4  and 5  show  the result  of  trying  to  decrypt a  short statement with and without Key2.  It still appears to be a weakness, however, the probability of an attacker having somehow accessed all but key2 is negligible and if it were the case decryption is more a matter of time rather than possibility.

In [97]:
# Our small data set
Input = ['Sussex','Sussex','Internationalization','Internationalization','Im going to brexit','Im going to brexit']
Key2 = [[0,5],'Unknown',[2,4,1,4,16],'Unknown',[9,15],'Unknown']
Encrypted_Message = ['CXSYJB','CXSYJB','PTDHBNEVNOPIPKJHDMVT','PTDHBNEVNOPIPKJHDMVT','RPZYPRJTJNCGDRIXKUFELW','RPZYPRJTJNCGDRIXKUFELW']
Decrypted_Message = ['sussex','ussex','internationalization','inenationaliaaion','imgoingtobrexit','imgoingobreit']
# Create dataframe
df = pd.DataFrame(Input)
df.columns = ['Input']
df['Key2'] = Key2
df['Encypted Message'] = Encrypted_Message
df['Decypted Message'] = Decrypted_Message
df

,Input,Key2,Encypted Message,Decypted Message
0,Sussex,"[0, 5]",CXSYJB,sussex
1,Sussex,Unknown,CXSYJB,ussex
2,Internationalization,"[2, 4, 1, 4, 16]",PTDHBNEVNOPIPKJHDMVT,internationalization
3,Internationalization,Unknown,PTDHBNEVNOPIPKJHDMVT,inenationaliaaion
4,Im going to brexit,"[9, 15]",RPZYPRJTJNCGDRIXKUFELW,imgoingtobrexit
5,Im going to brexit,Unknown,RPZYPRJTJNCGDRIXKUFELW,imgoingobreit


### Role Of Key3

Key 3 is a dictionary, keys are the index of the position in the Plain-text that a white-space was encountered, the values held in the key are the number of random characters that were added to the encrypted message.  So far we have restricted the values to a maximum of 3. The cipher could,theoretically, but not practically, store a billion random characters for each white-space.  Of course,that  would  be  a  rather  foolish  thing  to  do  here,  so  the  role  of  Key3  will  be  demonstrated  using  a maximum of 150 random characters for each white-space on the sentence:  ’if light has no mass, whyis it affected by the gravitational force of a black hole?’.

Encrypted Message:
    
    KHNDCYBVICAJMGYIJVADPP ZVXBXPBZLTREEMAINGXYQVWXKRKAYZMZSOYNADOAQSUECQUGNNKKHWREUCCFFKOLPEXOBOFTYGVCETMN GOOPTNIBUQLDHKZWMLK-WZTDCJCWGMSSFISHRGOSB QTDMNYFXFZOZTLFSHLPKFUWQJFEQXVOCPUDZORWI JS-RHEHLHLAENZQPQEWRANEGNKLFVIGMKBMSKURIM TQDMYHHRKKMCBPHQGBNDZOCWSE-JBUITNDRIKWTVA BTRYBXPXFSYTZPCWMMBZFWMZQYPKABCEXNKSWGUH HHDLEX-UFBUECGPBSVCVPHGNVBUHVOPHTYJDWEYRI JKPOUPDIUBDWKKUUBTCLNOHLWCGET-ZBDUNKLGMBD JSXAFJJOTWCEDIFKVAQQYEGOTRHFUHCHJEFVMMWX LEVXYOBUOI-WXEVPZOIRIKKWCMGSVVVLKBQEUGTFA YXLTEBOHNNQCDDBBLBPCJWNXHRKFYX-CBSUKRMXIM BQXPVUPAIUSMJVNPXBRAKIKAQVRLCLCJCZAVJIAA ZWAITADIAZYTLIOIPVSJFZYXS-FTINWGBVVCGNLBT HHEQHITEVLVIDJOTHOGHXDGWCYVEQOBDSJNNPFES GXUBKWN-HBTZGVWRPPFCNTOUFDGLQQBKVHQVCHDKI VMDHUXIOHRROHOOISDCTJXOPWXVQHK-JELBMVGGHE LQNNLVNRSFYRHCXEGUCNRSGXDOZCZEIBZNMPUJMW LQQTRNMMVGO-QVHQLDADFJMFCPBRROIKFCVMIWDNE CNHTAJARAWRJQPE
    
    
Key 3:
        defaultdict(¡class ’list’¿, 2:  68, 8:  49, 12:  66, 15:  44, 20:  33, 21:  52, 25:  17, 28:  39, 31:  12, 40:  54, 43:60, 47:  46, 61:  53, 67:  29, 70:  28, 72:  57, 78:  8)
        
        
        
Decryption using key 3:

    iflighthasnomasswhyisitaffectedbythegravitationalforceofablackhole
    
    
Decryption Without Key 3:

    ifjdwmhccjdteobnlzsoobteqohdlet  ytoojrevwhqntlookldckndkibfarpc  qcdbnljulfamteteqkfnojmkfllefivwjjcnytgrhlndgohoegjaqnjktsumxp lcqcokntnejstmprbrfqwccpoccecja nwohhdwlkbbefeatgcmhejjulbjqlhrcfpgkjaggqakaufpnchufrfqiepqso  ktvlxcqqkyppjjqaphxouphdxhdptbl  cacbjaxrdldcaipqqicelrbrfnfohnrgartkdhilorntbphhplmjlkhiudttar ibghbfsuaionpbadqomybegkqhnecaq kksxicnoxonlfrookxmerficnbmalsscflcsctcqoktleielbiihiwhqeoftqq qkokplbkqmspgbpqkfvknpqaebhlmme jafcwnahastdkdxutiggovmccwtonoglsgtnfnfqecladrcrlsgantbhdesbpw taqljinhicbnsniujhrruwlimceohpd bclnobuhkoebdohbfqxfmiqobhlfnmevmlwutijjejqbajojnitfirfskesbnl pogubgqbzbwidknekuhqnrjelmplnql ebbjjdgarlhfhaqfdubjelkjktmali



# The Cipher

In [98]:
class Cipher():
    
    
    def __init__(self,Max):
        self.Max_Random = Max
    
    # encodes a message and generates a private key
    def Encode(self,Message):

        Finished = False

        # Create a private key
        Key1 = Cipher_Create_Key(Message)
        # Create a private key
        Key2 = []
        # Create a private key
        Key3 = collections.defaultdict(list)

        # Create empty string for encrypted message
        Encrypted_Message = '';

        # Global count for loop termination
        Count =0;

        # Keep looping until message is fully encrypted
        while (not Finished):

            # For each element in the key
            for x in range(len(Key1)):

                # If we reach the end of the message exit loops
                if (Count == len(Message)):
                    Finished = True
                    break
                # Ignore spaces
                try:
                    # Replace the plaintext character with the letter n places to the right in the alphabet
                    # Where n is the integer at the xth position in the key. Wraparound if needed.
                    New_Position = int(Plaintext_Alphabet[Message[Count].upper()]) + int(Key1[x])

                    # If new position is greater than alphabet length then wraparound
                    if (New_Position > 25):
                        # Need to record positions where a wraparound happens
                        Key2.append(Count)
                        New_Position = -25 + (int(Plaintext_Alphabet[Message[Count].upper()]) + int(Key1[x]))

                    # Loop alphabet characters until we find a matching value
                    for Character in Plaintext_Alphabet.keys():
                        # If the index matches our new position
                        if (Plaintext_Alphabet[Character] == New_Position):
                            Encrypted_Message = Encrypted_Message + str(Character)
                            Count = Count+1
                            break

                except TypeError:
                    # Random int to determine the number of random characters to add for this space
                    Num_Randoms = random.randint(1,self.Max_Random)
                    # Change the above range to include more/less Random characters & whitespace

                    # Record where a space was and how amny random characters were added
                    Key3[Count] = Num_Randoms-1
                    for x in range (Num_Randoms):
                        # Generate random int for random character
                        New_Position = random.randint(0,25)
                        for Character in Plaintext_Alphabet.keys():
                            # If the index matches our new position
                            if (Plaintext_Alphabet[Character] == New_Position):
                                Encrypted_Message = Encrypted_Message + str(Character)
                    Count=Count+1
                    break

        return Key1,Key2,Key3,Encrypted_Message



    def Decrypt(self,Key1,Key2,Key3,Message):

        Decrypted = False
        Count = 0
        Key_Count =0


        Decrypted_Message =''
        # If the message wasn't encrypted by this cipher
        try:
            while (not Decrypted):


                if (Count == len(Message)):
                    Decrypted = True
                    break

                # For each element in the key
                for x in range(len(Key1)):

                    # If we reach the end of the message exit loops
                    if (Count == len(Message)):
                        Decrypted = True
                        break

                    # If a wraparound occured at this index
                    if (Key_Count in Key2):
                        New_Position = (25+ int(Plaintext_Alphabet[Message[Count].upper()])) - int(Key1[x])

                    # If a space was present in the original message
                    elif (Key_Count in Key3):

                        Decrypted_Message = Decrypted_Message + ''
                        Count = Count + Key3[Key_Count]+1
                        Key_Count = Key_Count+1
                        break

                    else:
                        # If no wraparound then the true character is just the current position - the key
                        New_Position = int(Plaintext_Alphabet[Message[Count].upper()]) - int(Key1[x])

                    # Loop alphabet characters until we find a matching value
                    for Character in Plaintext_Alphabet.keys():
                        # If the index matches our new position
                        if (Plaintext_Alphabet[Character] == New_Position):
                            Decrypted_Message = Decrypted_Message + str(Character).lower()
                            break

                    Count = Count+1
                    Key_Count = Key_Count +1

        except (AttributeError,IndexError): return ("That Was'nt Encrypted By Me!")


        return Decrypted_Message


    def Create_Key(self,Message):

        # initialize empty string for key
        New_Key =''
        # Hash the current UNIX time and remove the negation
        Key = str(hash(str("%.20f" % time.time())))[1:len(str(hash(str("%.20f" % time.time()))))]
        # add the hash of the message and then convert back to a string
        Key = int(Key)+ int(hash(Message))
        Key = str(Key)
        # Remove the decimal place and dash in the time sequence
        for Element in Key:

            if Element != '.' and Element !='-':
                New_Key += Element

        return New_Key


In [99]:
x = Cipher(150)
Key1,Key2,Key3,Encrypted_Message = x.Encode('if light has no mass, why is it affected by the gravitatioanal force of a black hole')
print('Encrypted Message:\n',Encrypted_Message,'\n')
print(time.time())
print(Key1)
print(Key2)
print(Key3)
x.Decrypt(Key1,Key2,Key3,Encrypted_Message)

Encrypted Message:
 RKNMGNVZGVUHZIHQKGPBYSAHIQLLXFILWXTBWFWLPXOXHVVYDOSIVUNLHCEBZREHUACPIVXAWKJZXTFUIXBGOJVWPATVEGJZKLBXQOYTIMIOWHUZCIEXITUOUJBMCYEYDYCLNZDYICZMTZZOXCWPEHMWQXQFXVKTYSOTCEWIPISRUFEJVAJQXQMMCFLACAKWQAAOTDLHOJWUNMQAEYNCJMOSWGNANMRFNTHGNLZZUEXXAQJQSYYXAXWTIKVDSJSFIZVVQRBWGOTYURZSMKQVTSUVFXSNXWRJUGMPDHLNKIWKOVCFQCOECLBZAHEZLBYZLJVATSQTXITAXOVTXUVPOKVLKCCCLFMLZHZEPJHHZRDDJVDJJAETQHXGUFFTHLMLULBPYPKYLGMEJAIOSOWDTFFHNCPOAMORXPSNCLYTVAIUGSJGMDWCRXUNUIJSJHMGVDCGEXNVTQFNTSCBBPBAACALXFMEPXOHCIOHLRJZKNTJZRYVWLGEITIZKFYIYQBGTRWMWKXZYXHKPHVHCKOICXFQLOBLRYBWVIXSYVSVHXABVYPYSYMOUSEZLJDEFTFWOXROZJHEWSYQPPLXNQIMMPFNFDENCLKUYXCCMRCVZSUAJHNNGFSPUVKOCZNPSPCBRBJKKEKBLEVGJGGOGQDXWBWXPQTVCQWENSNHSFQZFKVUMDXVDZXXVOMJAHACPFYRBPJLSAWVKEFWDCWULNDZVGVDMJLDWEAKFQXHMBGXFSVBOJQXXBWGCDSJWYXOXIEPFWNQWVHBUBEGMFSPHJEMYQYMUMSYTNPWFVQBHUJWSATQIRCKGGLHCJTYPZETUBYNHQGMMJHDQMSWKIMDMXTGXYROXXJJFXTNDJLZPZBKXYYZZEHOLZTGJEIALCVDNSLFLUJRHACVEKEXEGAQBTAZHMQCNPXOFSJXKOTWCMYQTEBUGMVXQZKKKFQVRZMYZXKWIXCKUCRNGKXMFGQTNZRB

'iflighthasnomasswhyisitaffectedbythegravitationalforceofablackhole'